In [1]:
import numpy as np
import numpy.random as ndm
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
from data_generator import generate_case_3
from iteration_deep import Est_deep
from I_spline import I_S
from Least_FD import LFD
from iteration_linear import Est_linear
from iteration_additive import Est_additive
from sklearn.model_selection import train_test_split
import dfply

Set seed

In [2]:
def set_seed(seed):
    np.random.seed(seed) 
    torch.manual_seed(seed) 

set_seed(8)

initial values

In [3]:
tau = 10
p = 3 
Set_n = np.array([500, 1000, 2000])
corr = 0.5 
n_layer = 3
n_node = 50
n_epoch = 200 #
Set_lr = np.array([2.8e-4, 3.2e-4, 4.2e-4]) #learning rate
Beta = np.array([-0.5,1]) #regression parameter
zeta = 2 #change point
node_D = np.array([35, 30, 30])   #deep
lr_D = np.array([4e-4, 4e-4, 4e-4])

node_L = np.array([45, 45, 50])   #linear
lr_L = np.array([1e-3, 1e-3, 1e-3])

node_A = np.array([30, 30, 30])   #addictive
lr_A = np.array([3e-4, 3e-4, 3e-4])

B = 200  #repulications 

generate data

In [5]:
test_data = generate_case_3(200, corr, Beta, 2)
X_test = test_data['X']
g_true = test_data['g_X']
dim_x = X_test.shape[0]
print(test_data)

{'Z': array([1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
       0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32), 'X': array([[1.40708268e+00, 0.00000000e+00, 0.00000000e+00, 1.73825645e+00,
        0.000000

In [ ]:
Z = ndm.normal(loc=0, scale=1, size=10) #parametric
Z_2 = ndm.normal(loc=2, scale=1.5, size=10) #change point
Z_3 = np.vstack((Z, Z*(Z_2>zeta)))



Generate X

In [ ]:
corr=0.5
mean = np.zeros(5)
cov = np.identity(5)*(1-corr) + np.ones((5, 5))*corr
def multivariatet(mu,Sigma,N,M):
        d = len(Sigma)
        g = np.tile(np.random.gamma(N/2,1/2,M),(d,1)).T
        Z = np.random.multivariate_normal(np.zeros(d),Sigma,M)
        return mu + Z/np.sqrt(g/N)
    
X = multivariatet(mean,cov,5,5)
X = np.clip(X, 0, 2) 

g_X = np.sqrt(X[:,0]*X[:,1])/5 + X[:,2]**2*X[:,3]/4 \
        + np.log(X[:,3]+1)/3 + np.exp(X[:,4])/2 - 1.91\
          +(0.2*(np.sqrt(X[:,0]*X[:,1])/5 + X[:,2]**2*X[:,3]/4 \
        + np.log(X[:,3]+1)/3 + np.exp(X[:,4])/2)**2 -0.45)*(Z_2>zeta)
print(g_X)



In [ ]:
def gendata(n,zeta,r=5,corr=0.5):
    Z_2 = ndm.normal(loc=2, scale=1.5, size=n) #change point
    mean = np.zeros(r)
    cov = np.identity(r)*(1-corr) + np.ones((r, r))*corr
    def multivariatet(mu,Sigma,N,M):
            d = len(Sigma)
            g = np.tile(np.random.gamma(N/2,1/2,M),(d,1)).T
            Z = np.random.multivariate_normal(np.zeros(d),Sigma,M)
            return mu + Z/np.sqrt(g/N)
    
    X = multivariatet(mean,cov,r,n)
    X = np.clip(X, 0, 2) 

    g_X = np.sqrt(X[:,0]*X[:,1])/5 + X[:,2]**2*X[:,3]/4 \
            + np.log(X[:,3]+1)/3 + np.exp(X[:,4])/2 - 1.91\
            +(0.2*(np.sqrt(X[:,0]*X[:,1])/5 + X[:,2]**2*X[:,3]/4 \
            + np.log(X[:,3]+1)/3 + np.exp(X[:,4])/2)**2 -1.36)*(Z_2>zeta)
    return{'X': np.array(X, dtype='float32'),
          'g_X': np.array(g_X, dtype='float32'),
          'Z_2': np.array(Z_2, dtype = 'float32')
          }

print(gendata(5,zeta=2))

In [ ]:
train_data = gendata(1000,zeta=2)
X_train = train_data['X']
g_train = train_data['g_X']
Z_2_train = train_data['Z_2']
test_data = gendata(200, zeta=2)
X_test = test_data['X']
g_test = test_data['g_X']
Z_2_test = test_data['Z_2']

establish DNN

In [ ]:
n_layer = 3
n_epoch = 50
n_lr = 1e-3
#X_train, X_test, g_train, g_test, Z_2_train,Z_2_test = \
    #train_test_split(X, g_X, Z_2, test_size=0.2,random_state=42)

def g_DCP(train_data,X_test,Z_2_test,n_layer,n_node,n_lr,n_epoch):
    #determine the location
    
    Z_2_tra = train_data['Z_2']
    n1 = len(Z_2_tra)   #sample size of train data
    n2 = len(Z_2_test)   #sample size of test data
    locx_tra1 = np.where(Z_2_tra==1)  #Z_2>zeta in train data
    locx_tra0 = np.where(Z_2_tra==0)
    locx_tes1 = np.where(Z_2_test==1)  #Z_2>zeta in test data
    locx_tes0 = np.where(Z_2_test==0)

    #input data into torch.tensor
    X_train = torch.Tensor(train_data['X'])
    g_train_true = torch.Tensor(train_data['g_X'])
    Z_2_train = torch.Tensor(train_data['Z_2'])
    X_test = torch.Tensor(X_test)
    Z_2_test = torch.Tensor(Z_2_test)
    
    #for I(Z_2>zeta)=0
    X_train0 = X_train[locx_tra0]
    g_train_true0 = g_train_true[locx_tra0]
    Z_2_train0 = Z_2_train[locx_tra0]
    X_test0 = X_test[locx_tra0]
    Z_2_test0 = Z_2_test[locx_tra0]
     #for I(Z_2>zeta)=1
    X_train1 = X_train[locx_tra1]
    g_train_true1 = g_train_true[locx_tra1]
    Z_2_train1 = Z_2_train[locx_tra1]
    X_test1 = X_test[locx_tra1]
    Z_2_test1 = Z_2_test[locx_tra1]
    class DNNModel(torch.nn.Module): #set the neural network
        def __init__(self):
            super(DNNModel, self).__init__()
            layers = []
            layers.append(nn.Linear(5, n_node)) #linear layer
            layers.append(nn.ReLU()) #ReLU layer
            for i in range(n_layer):
                layers.append(nn.Linear(n_node, n_node))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(n_node, 1))
            self.model = nn.Sequential(*layers) #complete the neural network
        def forward(self, x):   #forward propagation
            y_pred = self.model(x)
            return y_pred
    model = DNNModel()
    optimizer = torch.optim.Adam(model.parameters(), lr=n_lr)

    for epoch in range(n_epoch):
            pred_g_X = model(X_train)
            loss = nn.MSELoss()  #can be defined as other loss
            optimizer.zero_grad() #reset the gradient as 0
            loss.backward()   #back propagation
            optimizer.step()  #renew the parameter
    
    #
    g_train0 = model(X_train0)
    g_train1 = model(X_train1)
    g_test0 = model(X_test0)
    g_test1 = model(X_test1)
    g_train = np.zeros(n1)
    g_test = np.zeros(n2)
    #return the two model to their location
    g_train[locx_tra0] = g_train0
    g_train[locx_tra1] = g_train1
    g_test[locx_tes0] = g_test0
    g_test[locx_tes1] = g_test1
    g_train = g_train[:,0].detach().numpy()
    g_test = g_test[:,0].detach().numpy()
    return {
        'g_train': g_train,
        'g_test': g_test
    }

Find the location of specific item, using np.where for the array

In [ ]:
zz = np.array([1,0,0,1,1,0,1])
#zz = torch.Tensor(zz)
#index = np.where(zz==1)
#print(index)
#zz = zz[index].detach().numpy()
print(len(zz))
print(zz[-1])

In [ ]:
aa = np.array([1,1,4,5,1,4])
b = []
for a in aa:
    b = np.append(b,a**2)
print(np.array(aa))
print(np.array(b))
print(np.max(b))
print(np.where(b==np.max(b)))
print(np.zeros(5))
ts = np.vstack((np.zeros(5),np.ones(5)))
print(ts)
p,n=ts.shape
print(ts.shape[0])

In [ ]:
Z = ndm.binomial(1, 0.5, n) #parametric
Z_2 = ndm.normal(loc=2, scale=1, size=n) #change point
Z_2 = np.clip(Z_2, 1, 3)
ZC = np.vstack((Z, Z*(Z_2>zeta)))
print(ZC)
Theta = [0,1]

print(np.mean(np.exp(ZC.T@Theta+[1,0,3,0.5,0])))
#data = generate_case_3(10,0.5,Theta = [0,1],zeta=2)


In [2]:

import numpy as np
import torch
import pandas as pd
#import matplotlib.pyplot as plt
from data_generator import generate_case_3
from g_deep import g_D
from CCP_estimation import CCP_est
#from iteration_dcp import Est_dcp
from I_spline import I_S
from Least_FD import LFD
from g_dcp import g_DCP

In [3]:
tau = 10
p = 3 
n = 1000
corr = 0.5 
n_layer = 3
n_node = 50
n_epoch = 200 #
Set_lr = np.array([2.8e-4, 3.2e-4, 4.2e-4]) #learning rate
n_lr = 1e-4
Theta = [0, 1]
zeta = 2

#dim_x = X_test.shape[0]
u_value = np.array(np.linspace(0, tau, 50), dtype="float32") 
Lambda_true = np.sqrt(u_value)/5 
m = 10 
nodevec = np.array(np.linspace(0, tau, m+2), dtype="float32")

m0 = 4 
nodevec0 = np.array(np.linspace(0, 2, m0+2), dtype="float32")

node_D = np.array([35, 30, 30])   #deep
lr_D = np.array([4e-4, 4e-4, 4e-4])

c0 = np.array(0.1*np.ones(m+p), dtype="float32") 
Theta0 = np.array([0,0], dtype='float32')

In [4]:

u_value = np.array(np.linspace(0, tau, 50), dtype="float32") 
Lambda_true = np.sqrt(u_value)/5 
m = 10 
nodevec = np.array(np.linspace(0, tau, m+2), dtype="float32")

m0 = 4 
nodevec0 = np.array(np.linspace(0, 2, m0+2), dtype="float32")

node_D = np.array([35, 30, 30])   #deep
lr_D = np.array([4e-4, 4e-4, 4e-4])

c0 = np.array(0.1*np.ones(m+p), dtype="float32") 
Theta0 = np.array([0,0], dtype='float32')


train_data = generate_case_3(n, corr, Theta, zeta)
Z_train = train_data['Z']
U_train = train_data['U']
De_train = train_data['De']
g_train = train_data['g_X']
Z_2_train = train_data['Z_2']

zeta0 = np.mean(Z_2_train)

test_data = generate_case_3(200, corr, Theta, zeta)
X_test = test_data['X']
g_true = test_data['g_X']
Z_2_test = test_data['Z_2']


Z_t = torch.Tensor(Z_train)
Z_2_t = torch.Tensor(Z_2_train)
g_X_t = torch.Tensor(g_train)
De_t = torch.Tensor(De_train)
ZC_t = np.vstack((Z_train,Z_train*(Z_2_train>zeta0)))
ZC_t = torch.Tensor(ZC_t)
Theta = torch.Tensor(Theta)
Lambda_U = I_S(m,c0,U_train,nodevec)
Lambda_U_t = I_S(m,c0,U_train,nodevec)
Lambda_U_t = torch.Tensor(Lambda_U_t)
Lam1 = Lambda_U_t * torch.exp(ZC_t.T@Theta + g_X_t)
loss_fun = -torch.mean(De_t*torch.log(1-torch.exp(-Lam1)+1e-5) - (1-De_t)*Lam1)
print(loss_fun)
print(zeta0)
from g_deep import g_D
from CCP_estimation import CCP_est
from iteration_dcp import Est_dcp
from I_spline import I_S
from Least_FD import LFD
from g_dcp import g_DCP
from Theta_estimate import Theta_est
from zeta_estimate import zeta_est



#g_X = g_DCP(train_data,X_test,Z_2_test,Lambda_U_t,Theta,Theta0,zeta,n_layer,n_node,n_lr,n_epoch)


tensor(0.2769)
1.9909074


In [5]:

Z_2_tra = train_data['Z_2']
n1 = len(Z_2_tra)   #sample size of train data
n2 = len(Z_2_test)   #sample size of test data
locx_tra1 = np.where(Z_2_tra>zeta0)  #Z_2>zeta in train data
locx_tra0 = np.where(Z_2_tra<=zeta0)
locx_tes1 = np.where(Z_2_test>zeta0)  #Z_2>zeta in test data
locx_tes0 = np.where(Z_2_test<=zeta0)

Z_train = torch.Tensor(train_data['Z'])
X_train = torch.Tensor(train_data['X'])
U_train = torch.Tensor(train_data['U'])
De_train = torch.Tensor(train_data['De'])
g_train_true = torch.Tensor(train_data['g_X'])
Z_2_train = torch.Tensor(train_data['Z_2'])
Lambda_U0 = Lambda_U[locx_tra0]
Lambda_U1 = Lambda_U[locx_tra1]
X_test0 = X_test[locx_tes0]
X_test1 = X_test[locx_tes1]

#input data into torch.tensor
Z_train0 = torch.Tensor(Z_train[locx_tra0])
X_train0 = torch.Tensor(X_train[locx_tra0,:])
U_train0 = torch.Tensor(U_train[locx_tra0])
De_train0 = torch.Tensor(De_train[locx_tra0])
g_train_true0 = torch.Tensor(g_train_true[locx_tra0])
Z_2_train0 = torch.Tensor(Z_2_train[locx_tra0])
Lambda_U0 = torch.Tensor(Lambda_U0)
X_test0 = torch.Tensor(X_test0)

Z_train1 = torch.Tensor(Z_train[locx_tra1])
X_train1 = torch.Tensor(X_train[locx_tra1,:])
U_train1 = torch.Tensor(U_train[locx_tra1])
De_train1 = torch.Tensor(De_train[locx_tra1])
g_train_true1 = torch.Tensor(g_train_true[locx_tra1])
Z_2_train1 = torch.Tensor(Z_2_train[locx_tra1])
Lambda_U1 = torch.Tensor(Lambda_U1)   
X_test1 = torch.Tensor(X_test1)
#print('aasq')


Theta0 = torch.Tensor(Theta0)
#print(locx_tra1)
#print(Z_2_train[locx_tra1])
#print(X_train[locx_tra1,:])
#print(X_train)


C:\Users\txw\AppData\Local\Temp\ipykernel_31892\2321191037.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  X_train0 = torch.Tensor(X_train[locx_tra0,:])


In [6]:
#train DNN for Z_2<=zeta
class DNNModel0(torch.nn.Module): #set the neural network
    def __init__(self):
        super(DNNModel0, self).__init__()
        layers = []
        layers.append(nn.Linear(5, n_node)) #linear layer
        layers.append(nn.ReLU()) #ReLU layer
        for i in range(n_layer):
            layers.append(nn.Linear(n_node, n_node))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(n_node, 1))
        self.model = nn.Sequential(*layers) #complete the neural network
    def forward(self, x):   #forward propagation
        y_pred = self.model(x)
        return y_pred

model = DNNModel0()
optimizer = torch.optim.Adam(model.parameters(), lr=n_lr)


def my_loss(De, Z, Theta, Lambda_U, g_X, Z_2, zeta):
    ZC = np.vstack((Z,Z*(Z_2>zeta)))
    ZC = torch.Tensor(ZC)
    Lam1 = Lambda_U * torch.exp(ZC.T@Theta + g_X)
    loss_fun = -torch.mean(De*torch.log(1-torch.exp(-Lam1)+1e-5) - (1-De)*Lam1)
    return loss_fun

for epoch in range(n_epoch):
    pred_g_X = model(X_train0)
    loss = my_loss(De_train0, Z_train0, Theta0, Lambda_U0, Z_2_train0, zeta0, pred_g_X[:, 0])
    loss.requires_grad_(True)
    optimizer.zero_grad() #reset the gradient as 0
    loss.backward()   #back propagation
    optimizer.step()  #renew the parameter

g_train0 = model(X_train0)
g_test0 = model(X_test0)
g_train0 = g_train0[:,0].detach().numpy()
g_test0 = g_test0[:,0].detach().numpy()
print(g_test0)
    


[0.06659405 0.06851879 0.07605243 0.06420349 0.06659405 0.07370904
 0.07478966 0.08151164 0.07544191 0.06514858 0.0653682  0.06913154
 0.06659405 0.06181982 0.06894178 0.07401966 0.07229486 0.06894496
 0.07214272 0.07086523 0.06477824 0.06659405 0.07485047 0.06930139
 0.07056303 0.07275105 0.07536077 0.06898375 0.07572173 0.06959607
 0.06434268 0.06898209 0.06624186 0.06773318 0.07598473 0.07104519
 0.07309871 0.06691195 0.06910893 0.076205   0.06982376 0.07638603
 0.06655115 0.07901655 0.06830037 0.07093181 0.06650609 0.06605513
 0.06397839 0.06718388 0.07698496 0.06727683 0.06659405 0.06659405
 0.08010241 0.06835021 0.07592168 0.06480147 0.0663399  0.0664574
 0.06659405 0.06659405 0.06659405 0.06756331 0.07763471 0.06932912
 0.07228421 0.07725938 0.06659405 0.06659405 0.06603593 0.06439888
 0.0750322  0.0762057  0.07558505 0.06840745 0.06760281 0.07808123
 0.06669836 0.06425573 0.06549397 0.07536077 0.06927495 0.06659405
 0.06659405 0.06434268 0.0684566  0.07556543 0.06714419 0.06601

In [7]:
#train DNN for Z_2>zeta
class DNNModel1(torch.nn.Module): #set the neural network
    def __init__(self):
        super(DNNModel1, self).__init__()
        layers = []
        layers.append(nn.Linear(5, n_node)) #linear layer
        layers.append(nn.ReLU()) #ReLU layer
        for i in range(n_layer):
            layers.append(nn.Linear(n_node, n_node))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(n_node, 1))
        self.model = nn.Sequential(*layers) #complete the neural network
    def forward(self, x):   #forward propagation
        y_pred = self.model(x)
        return y_pred

model = DNNModel1()
optimizer = torch.optim.Adam(model.parameters(), lr=n_lr)


    #def my_loss(De, Z, Theta, Lambda_U, g_X, Z_2, zeta):
        #ZC = np.vstack((Z,Z*(Z_2>zeta)))
        #Lam1 = Lambda_U * torch.exp(ZC.T@Theta + g_X)
        #loss_fun = -torch.mean(De*torch.log(1-torch.exp(-Lam1)+1e-5) - (1-De)*Lam1)
        #return loss_fun

for epoch in range(n_epoch):
    pred_g_X = model(X_train1)
    loss = my_loss(De_train1, Z_train1, Theta0, Lambda_U1, Z_2_train1, zeta0, pred_g_X[:, 0])
    loss.requires_grad_(True)
    optimizer.zero_grad() #reset the gradient as 0
    loss.backward()   #back propagation
    optimizer.step()  #renew the parameter

g_train1 = model(X_train1)
g_test1 = model(X_test1)
g_train1 = g_train1[:,0].detach().numpy()
g_test1 = g_test1[:,0].detach().numpy()
    

#reorganize the data
g_train = np.array(np.zeros(n1))
g_train[locx_tra0] = g_train0
g_train[locx_tra1] = g_train1

g_test = np.array(np.zeros(n2))
g_test[locx_tes0] = g_test0
g_test[locx_tes1] = g_test1

print(g_test1)

print(g_test)

[0.12591355 0.12669073 0.1341394  0.1306205  0.11722872 0.12826374
 0.12627444 0.12880734 0.10906009 0.12428268 0.13195208 0.13202804
 0.12963593 0.12428268 0.1223175  0.13131829 0.15154886 0.12428269
 0.12524717 0.1488354  0.13270967 0.12817612 0.11344585 0.13578539
 0.12045701 0.12488264 0.12428269 0.12428268 0.12409344 0.12406356
 0.13737212 0.12242812 0.11956942 0.12726966 0.1290403  0.12497766
 0.12579325 0.12428268 0.13346875 0.1292221  0.12889557 0.12710352
 0.14211786 0.13345212 0.13028112 0.12157978 0.14624862 0.13011797
 0.1488354  0.13068174 0.13339972 0.12893032 0.12096152 0.12428269
 0.13340841 0.12436281 0.12461098 0.12190942 0.12621816 0.10953461
 0.12428269 0.12525703 0.13499019 0.11834127 0.12428268 0.12612283
 0.12420039 0.12892649 0.11732037 0.11107737 0.1340236  0.12428269
 0.12477808 0.13041481 0.13501124 0.12814245 0.12199256 0.12500899
 0.13738795 0.11408579 0.12779143 0.12655959 0.12428268 0.11189784
 0.12490163 0.12767398 0.13815051 0.12428269 0.11270965 0.1259

In [ ]:
seq=0.05
Z_min = np.min(Z_2)
Z_max = np.max(Z_2)
num = np.floor((Z_max-Z_min)/seq)
zeta_grid = np.linspace(Z_min, Z_max, num)  #the search grid

#define the log-likelihood loss
def BZ2(*args):
    ZC = np.vstack((Z,Z*(Z_2>args[0])))
    Lam = Lambda_U * np.exp(ZC.T @ Theta + g_X)
    Loss_F = np.mean(-De * np.log(1 - np.exp(-Lam) + 1e-5) + (1 - De) * Lam)
    return Loss_F
    
zeta_loss = np.array([])
for zeta in zeta_grid:
    zeta_loss = np.append(zeta_loss, BZ2(zeta))
loss_min = np.min(zeta_loss)
loc = np.where(zeta_loss==loss_min)
zeta_est = zeta_grid[loc]